In [3]:
import os
import sys

from dotenv import load_dotenv

load_dotenv("../../_apikeys.env")
api_key = os.getenv("DoogieOpenaiKey")
os.environ['OPENAI_API_KEY'] = api_key

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
#from langchain.vectorstores import Milvus
#from langchain.vectorstores import OpenAIVectorStore

In [5]:
#
# Step 1
#
import urllib.request

urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="06_07_test.pdf")

('06_07_test.pdf', <http.client.HTTPMessage at 0x1166c1370>)

In [5]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader( "06_07_test.pdf" )
pages = loader.load_and_split()  # about 17s
#pages = loader.load()  # about 16s, include empty page


In [6]:
print( type(loader), type(pages), type(pages[0]), type(pages[0].page_content))
print(  sys.getsizeof(loader), len(pages), sys.getsizeof(pages[0]), len(pages[0].page_content) )
print( pages[0] )
print( pages[0].page_content)
print( "example:", pages[6].page_content[:50] )

<class 'langchain_community.document_loaders.pdf.PyPDFLoader'> <class 'list'> <class 'langchain_core.documents.base.Document'> <class 'str'>
48 445 72 57
page_content='2023
북한인권보고서
2023 Report on North Korean Human Rights
통일부' metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2023-07-31T13:50:27+09:00', 'moddate': '2023-07-31T13:57:54+09:00', 'trapped': '/False', 'source': '06_07_test.pdf', 'total_pages': 448, 'page': 0, 'page_label': '1'}
2023
북한인권보고서
2023 Report on North Korean Human Rights
통일부
example: 발간사
07
끝으로, 북한인권의 개선을 위해 북한에서의 고통스러웠던 인권
침해 경험을 들려


In [7]:
print( 'pages Max:', max(len(apage.page_content) for apage in pages ) )
print( 'pages Min:', min(len(apage.page_content) for apage in pages ) )
print( 'pages Sum:', sum(len(apage.page_content) for apage in pages ) )
print( '#of chunks:', len(pages) )
print( 'Chunk Avg:', sum(len(apage.page_content) for apage in pages ) / len(pages) )

pages Max: 1640
pages Min: 6
pages Sum: 372229
#of chunks: 445
Chunk Avg: 836.4696629213483


In [ ]:
print( pages[442].page_content )

In [8]:
# Step 2: case1:
# RecursiveCharacterTextSpilter를 사용함
#
from langchain.text_splitter import RecursiveCharacterTextSplitter

textSplitter = RecursiveCharacterTextSplitter(
    chunk_size= 1000,
    chunk_overlap=0,
    separators=["\n\n", "\n", " ", ""]
)

splitedDocs = textSplitter.split_documents( pages )
# 48 496 72 57, at chunk=1000
# 48 445 72 57, at chunk=2000
# 48 445 72 57, at chunk=3000

In [9]:
print( type(textSplitter), type(splitedDocs), type(splitedDocs[0]), type(splitedDocs[0].page_content) )
print( sys.getsizeof(textSplitter), len(splitedDocs), sys.getsizeof(splitedDocs[0]), len(splitedDocs[0].page_content) )
print( splitedDocs[0].page_content)
print( "example:", splitedDocs[0].page_content[:50] )

<class 'langchain_text_splitters.character.RecursiveCharacterTextSplitter'> <class 'list'> <class 'langchain_core.documents.base.Document'> <class 'str'>
48 496 72 57
2023
북한인권보고서
2023 Report on North Korean Human Rights
통일부
example: 2023
북한인권보고서
2023 Report on North Korean Human Rig


In [11]:
#
# tmp, check view
# refer: with direct list, splitedDocs
print( 'pages Max:', max(len(apage.page_content) for apage in splitedDocs ) )
print( 'pages Min:', min(len(apage.page_content) for apage in splitedDocs ) )
print( 'pages Sum:', sum(len(apage.page_content) for apage in splitedDocs ) )
print( '#of chunks:', len(pages) )
print( 'Chunk Avg:', sum(len(apage.page_content) for apage in splitedDocs ) / len(splitedDocs) )


pages Max: 1000
pages Min: 6
pages Sum: 372148
#of chunks: 445
Chunk Avg: 750.2983870967741


In [12]:
#
# tmp, check view
# refer: Chunk Avg with map() method....
#
chunks = [ splitedDocs.page_content for splitedDocs in splitedDocs ]
print( len(chunks), type(chunks) )

print( 'Chunk Max:', max(len(achunk) for achunk in chunks ) )
print( 'Chunk Min:', min(len(achunk) for achunk in chunks ) )
print( 'sumChunks:', sum(len(achunk) for achunk in chunks ) )
print( '#of chunk:', len(chunks) )
print( 'Chunk Avg:', sum(len(achunk) for achunk in chunks ) / len(chunks) )
print( 'Chunk Avg:', sum(map( len, chunks )) / len(chunks) )

for i in range( 5 ):
    print( "splitedDocs[", i, "]=", len(splitedDocs[i].page_content), )
print( len(splitedDocs))

496 <class 'list'>
Chunk Max: 1000
Chunk Min: 6
sumChunks: 372148
#of chunk: 496
Chunk Avg: 750.2983870967741
Chunk Avg: 750.2983870967741
splitedDocs[ 0 ]= 57
splitedDocs[ 1 ]= 60
splitedDocs[ 2 ]= 53
splitedDocs[ 3 ]= 512
splitedDocs[ 4 ]= 468
496


In [ ]:
#
# tmp, check view. all list
#
for i in range( len( splitedDocs )) :
    print( "splitedDocs[{}]".format( i ),
           "len=", len(splitedDocs[i].page_content),
           "ctnt=", splitedDocs[i].page_content[:10], "**", splitedDocs[i].page_content[-10:] )

In [ ]:
# Step 2: case2:
# SemanticChunker를 사용함
#
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

embeddings = OpenAIEmbeddings()
print("current OpenAIEmbeddings().model=", embeddings.model)
textsplitter = SemanticChunker(
    OpenAIEmbeddings(model="text-embedding-3-large"),
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95,
)

splitedDocs = textSplitter.split_documents( pages )

print( type(textSplitter), type(splitedDocs), type(splitedDocs[0]), type(splitedDocs[0].page_content) )
print( sys.getsizeof(textSplitter), len(splitedDocs), sys.getsizeof(splitedDocs[0]), len(splitedDocs[0].page_content) )
print( splitedDocs[0].page_content)
print( "example:", splitedDocs[0].page_content[:50] )

In [128]:
#
# tmp, check OpenAI model
#
#from langchain.embeddings import OpenAIEmbeddings
#from langchain.openai import OpenAIEmbeddings
#from langchain.vectorstores import Chroma

myEmbeddings = OpenAIEmbeddings()
print("current OpenAIEmbeddings().model=", myEmbeddings.model)
myEmbeddings = OpenAIEmbeddings(model="text-embedding-3-large")
print("current OpenAIEmbeddings().model=", myEmbeddings.model)


current OpenAIEmbeddings().model= text-embedding-ada-002
current OpenAIEmbeddings().model= text-embedding-3-large


In [143]:
#
# Step3: too long splitedDocs
#
from langchain.embeddings import OpenAIEmbeddings
#from langchain.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

myEmbeddings = OpenAIEmbeddings(model="text-embedding-3-small")
print("current OpenAIEmbeddings().model=", myEmbeddings.model)

vdb = Chroma.from_documents(splitedDocs, myEmbeddings)
print("vdb적재문서수=", vdb._collection.count())


current OpenAIEmbeddings().model= text-embedding-3-small


BadRequestError: Error code: 400 - {'error': {'message': 'Requested 367501 tokens, max 300000 tokens per request', 'type': 'max_tokens_per_request', 'param': None, 'code': 'max_tokens_per_request'}}

In [46]:
#
# chatgpt guide
#
import shutil, os
from langchain.embeddings import OpenAIEmbeddings
#from langchain_chroma.vectorstores import Chroma
from chromadb.config import Settings
from langchain_chroma import Chroma

settings = Settings(
    is_persistent=True,
    allow_reset=True
)

myEmbeddings = OpenAIEmbeddings(model="text-embedding-3-small")
persist_dir = "my_chroma_db12"

def process_in_batches(documents, batch_size=100, reset_db=True):
    global vdb
    try:
        vdb._client.reset()
        del vdb
    except:
        pass
    # --- 1. 기존 DB 디렉토리 삭제 (옵션) ---
    if reset_db and os.path.exists(persist_dir):
        shutil.rmtree(persist_dir)
        os.makedirs(persist_dir, exist_ok=True) # 확실히 쓰기 가능하게 생성
        print(f"[INFO] '{persist_dir}' 초기화 완료")

    # --- 2. 새로운 Chroma DB 객체 준비 ---
    vdb = Chroma(
        client_settings=settings,
        collection_name="my_collection",
        embedding_function=myEmbeddings,
        persist_directory=persist_dir
    )

    # --- 3. 배치 단위로 문서 추가 ---
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        vdb.add_documents(batch)
        print(f"[INFO] 처리된 문서: {i + len(batch)}/{len(documents)} "
              f"(현재 총 {vdb._collection.count()} 개)")

    return vdb

# 사용 예시
vdb = process_in_batches(splitedDocs, batch_size=100, reset_db=True)
print("최종 vdb 문서 수 =", vdb._collection.count())

# Chroma 객체 사용 후 반드시 닫기
if vdb is not None:
    try:
        vdb._client.close()  # langchain_chroma에서 사용 가능
    except Exception as e:
        print("close 실패:", e)
    vdb = None


[INFO] 'my_chroma_db12' 초기화 완료


InternalError: Query error: Database error: error returned from database: (code: 1032) attempt to write a readonly database

In [33]:
#
#
#
print( vdb._chroma_collection.configuration_json )
print( vdb._chroma_collection.count() )
print( vdb._collection.count() )
print( vdb._collection.name )
print( vdb._client.list_collections() )
print( vdb._chroma_collection.name )

{'hnsw': {'space': 'l2', 'ef_construction': 100, 'ef_search': 100, 'max_neighbors': 16, 'resize_factor': 1.2, 'sync_threshold': 1000}, 'spann': None, 'embedding_function': None}
496
496
my_collection
[Collection(name=my_collection)]
my_collection


In [38]:
#
# Step3: divided chunking, with batch_size=100
#
import shutil
import os
from langchain.embeddings import OpenAIEmbeddings
#from langchain.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

myEmbeddings = OpenAIEmbeddings(model="text-embedding-3-small")
print("current OpenAIEmbeddings().model=", myEmbeddings.model)

chroma_persist_dir = "my_chroma_db"

if os.path.exists(chroma_persist_dir):
    shutil.rmtree(chroma_persist_dir)

vdb = Chroma(
    embedding_function=myEmbeddings,
    persist_directory=chroma_persist_dir
)

print("vdb적재문서수a=", vdb._collection.count() )
vdb._client.delete_collection(vdb._collection.name)
#print("vdb적재문서수b=", vdb._collection.count() if vdb is not None else -1 )
vdb = process_in_batches(splitedDocs)
print("vdb적재문서수z=", vdb._collection.count() )

def process_in_batches(documents, batch_size=100):

    print("in func, vdb size:",vdb._collection.count() if vdb is not None else -1)

    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        vdb.add_documents(batch)
        print(f"처리된 문서: {i + len(batch)}/{len(documents)}", vdb._collection.count())
    return vdb

current OpenAIEmbeddings().model= text-embedding-3-small


InternalError: Database error: error returned from database: (code: 1032) attempt to write a readonly database

In [ ]:
print( len( vdb ), type( vdb ) )

In [2]:
## exception sample

try:
    import chromadb
except Exception as e:
    print("CHROMADB LOAD ERROR:", repr(e))


try:
    import chromadb
except Exception as e:
    import traceback
    traceback.print_exc()